In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain import hub
from dotenv import load_dotenv
from langchain.tools import BaseTool
import os

load_dotenv()


True

In [2]:
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("azure_openai_deployment", ""), model="gpt-4o-mini"
)
db = SQLDatabase.from_uri("sqlite:///data/library_database.sqlite")


In [37]:
books_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Book": {"properties": {"isbn": {"description": "ISBN of the book", "title": "Isbn", "type": "string"}, "title": {"description": "title of the book", "title": "Title", "type": "string"}, "category": {"description": "category of the book", "title": "Category", "type": "string"}, "author": {"description": "author of the book", "title": "Author", "type": "string"}, "publisher": {"description": "publisher of the book", "title": "Publisher", "type": "string"}, "price": {"description": "rental price of the book", "title": "Price",

In [34]:
from typing import Optional
import json
from output_parsers.books import books_parser

class GetBooksTool(BaseTool):
    name:str = "get_books"
    description: str = (
        "Get a list of all books from the database. "
        "You can optionally filter by book title or category. "
        "The input should be a JSON string with the following keys"
        "'book_title', 'category', 'author', 'publisher', 'min_price','max_price'"
        "If you have no data for min_price and max_price, don't include them in the JSON string"
        f"Format instructions: {books_parser.get_format_instructions()}"
    )
    
    def _run(self, json_data:str):
        data = json.loads(json_data)
        book_title = data.get("book_title", "")
        category = data.get("category", "")
        author = data.get("author", "")
        publisher = data.get("publisher", "")
        min_price = data.get("min_price", 0)
        max_price = data.get("max_price", 1000)
        
        return self.run_query(book_title, category, author, publisher, min_price, max_price)

    def run_query(self, 
            book_title: Optional[str] = "", 
            category: Optional[str] = "",
            author: Optional[str] = "",
            publisher: Optional[str] = "",
            min_price: Optional[int] = 0, 
            max_price: Optional[int] = 1000):
        # Query the books table
        query = f"""SELECT * FROM books WHERE lower(book_title) LIKE '%{book_title.lower()}%' 
            AND category LIKE '%{category.lower()}%' 
            AND lower(author) LIKE '%{author.lower()}%' 
            AND lower(publisher) LIKE '%{publisher.lower()}%' 
            AND rental_price BETWEEN {min_price} AND {max_price}
            """
        results = db.run(query)

        return results

In [29]:
class CheckAvailabilityTool(BaseTool):
    name:str = "check_availability"
    description:str = (
        "Check if a book is available for rent." 
        "In order to be available, the book most exist in the database and currently not rented out."
        "The input should be isbn"
    )
    
    def _run(self, isbn: Optional[str] = ""):
        if isbn:
            # Query the rentals table
            result = db.run(f"""
                SELECT * FROM rentals WHERE isbn='{isbn}' AND return_date IS NULL;
            """)

            return "Available" if not result else "Not Available"

        return "Not found"

In [30]:
from datetime import datetime

In [31]:
customer_id = 2
class RentBookTool(BaseTool):
    name:str = "rent_book"
    description:str = (
        "Rent a book to a user if the book is available for rent."
        "Use global customer_id variable and take isbn as input variable."
    )

    def _run(self, isbn: str):
        # Insert a new rental record
        today = datetime.now().strftime("%d.%m.%Y")
        db.run(f"""
            INSERT INTO rentals (isbn, customer_id, rent_date)
            VALUES ('{isbn}', {customer_id}, '{today}')
        """)
        return "Book rented successfully" 

In [35]:
from langchain.agents import initialize_agent, Tool

tools = [GetBooksTool(),CheckAvailabilityTool(),RentBookTool()]
agent = initialize_agent(tools,llm,verbose=True)

In [36]:
output = agent.invoke("Can you list the history books in your library?")



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'"properties"', 'input', '"$defs"'}

In [20]:
import pprint
pprint.pprint(output['output'])

('Here are the history books available in the library:\n'
 '\n'
 '1. **The Histories** by Herodotus (ISBN: 978-0-14-044930-3) - $5.50 '
 '(Publisher: Penguin Classics)\n'
 '2. **The Guns of August** by Barbara W. Tuchman (ISBN: 978-0-19-280551-1) - '
 '$7.00 (Publisher: Oxford University Press)\n'
 '3. **Sapiens: A Brief History of Humankind** by Yuval Noah Harari (ISBN: '
 '978-0-307-58837-1) - $8.00 (Publisher: Harper Perennial)\n'
 '4. **The Diary of a Young Girl** by Anne Frank (ISBN: 978-0-375-41398-8) - '
 '$6.50 (Publisher: Bantam)\n'
 "5. **A People's History of the United States** by Howard Zinn (ISBN: "
 '978-0-393-05081-8) - $9.00 (Publisher: Harper Perennial)\n'
 '6. **Guns, Germs, and Steel: The Fates of Human Societies** by Jared Diamond '
 '(ISBN: 978-0-393-91257-8) - $7.00 (Publisher: W. W. Norton & Company)\n'
 '7. **1491: New Revelations of the Americas Before Columbus** by Charles C. '
 'Mann (ISBN: 978-0-7432-7357-1) - $6.50 (Publisher: Vintage Books)')


In [21]:
from langchain.output_parsers import S

parser = MarkdownListOutputParser()
parsed_output = parser.parse(output['output'])

print(parsed_output)

[]
